Importing the libraries

In [133]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms


ModuleNotFoundError: No module named 'torchvision'

Uploading the dataset and deciding it into train/validation/test sets

In [134]:
# Assuming images are in a folder named "dataset" and labeled by subfolders
def load_images_from_folder(folder):
    images = []
    labels = []
    label = 0

    for subdir in os.listdir(folder):
        subfolder = os.path.join(folder, subdir)
        if os.path.isdir(subfolder):
            for filename in os.listdir(subfolder):
                img_path = os.path.join(subfolder, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    images.append(img)
                    labels.append(label) # you can append(subdir) if you want to have string labels
            label += 1
    return images, labels

images, labels = load_images_from_folder("Dataset")


# Split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, random_state=42, stratify=labels)

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# converting our lists to np arrays
X_train, X_val, X_test, y_train, y_val, y_test = np.array(X_train,dtype=float), np.array(X_val,dtype=float), np.array(X_test,dtype=float), np.array(y_train,dtype=float), np.array(y_val,dtype=float), np.array(y_test,dtype=float)

# print(images, labels)
print(X_train, X_val, X_test, y_train, y_val, y_test)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] [[[0. 0. 0

Data augmentation - I dont know if we need it?

In [135]:
# def augment_data(dataset, dataset_labels, augment=True):
#     augmented_image = []
#     augmented_image_labels = []
# 
#     for num, image in enumerate(dataset):
#         augmented_image.append(image)
#         augmented_image_labels.append(dataset_labels[num])
#         if augment:
#             # Creating a flipped version of the image
#             flipped = cv2.flip(image, 1)
#             augmented_image.append(flipped)
#             augmented_image_labels.append(dataset_labels[num])
# 
#             # Creating a rotated version of the image
#             height, width = image.shape[:2]
#             rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), 30, 1)
#             rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
#             augmented_image.append(rotated_image)
#             augmented_image_labels.append(dataset_labels[num])
# 
#     return np.array(augmented_image), np.array(augmented_image_labels)
# 
# X_train_augmented, y_train_augmented = augment_data(X_train, y_train)

Creating a data loader

In [136]:
# # Creating our own data loader
# def load_batch(images, labels, batch_size=32):
#     for i in range(0, len(images), batch_size):
#         yield images[i:i+batch_size], labels[i:i+batch_size]
# 
# # Example of using load_batch to get batches of data
# for X_batch, y_batch in load_batch(X_train, y_train): # if you augment for X_batch, y_batch in load_batch(X_train_augmented, y_train_augmented):
#     # Now you can use this batch of data to train your model
#     pass

# Using pytorch's Dataloader

In [137]:
# Assuming X_train and y_train are your preprocessed training data and labels
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).long()

# Creating a TensorDataset
train_dataset = TensorDataset(X_train, y_train)

# Defining the DataLoader with the dataset, batch size, and shuffle option
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Do the same for the validation
X_val, y_val = torch.tensor(X_val, dtype=torch.float32),torch.tensor(y_val, dtype=torch.long)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Do the same for the test
X_test, y_test = torch.tensor(X_test, dtype=torch.float32),torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Example of using DataLoader to get batches of data
for X_batch, y_batch in train_loader:
    # Now you can use this batch of data to train your model
    pass

Building the CNN

In [138]:
class AlzheimerCNN(nn.Module):
    def __init__(self):
        super(AlzheimerCNN, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 1, kernel_size=3, padding=1)

        # Fully connected layers
        # self.fc1 = nn.Linear(64 * 32 * 32, 256)
        self.fc1 = nn.Linear(32 * 32, 256)
        self.fc2 = nn.Linear(256, 4)  # Four classes to output

    def forward(self, x):
        # print(f"Input shape: {x.shape}")  # Add this line to print the input shape
    
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
    
        # print(f"After conv1 and pooling shape: {x.shape}")  # Add this line
    
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
    
        # print(f"After conv2 and pooling shape: {x.shape}")  # Add this line
    
        x = x.view(x.size(0), -1)
    
        # print(f"After flattening shape: {x.shape}")  # Add this line
    
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
    
        return F.log_softmax(x, dim=1)



Defining the loss function and optimize

In [139]:
model = AlzheimerCNN()

# Defining the loss function
criterion = nn.CrossEntropyLoss()

# Defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


Training the model

In [141]:
def train_model(model, criterion, optimizer, train_loader, epochs=1):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            print(f"Input shape: {inputs.shape}, Labels shape: {labels.shape}")  # Check shapes

            optimizer.zero_grad()

            outputs = model(inputs.float())
            print(f"Output shape: {outputs.shape}")  # Check output shape

            # Ensure labels are long type for the CrossEntropyLoss
            labels = labels.long()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            print(f"[{epoch+1}, {i+1}] loss: {running_loss / (i+1)}")

train_model(model, criterion, optimizer, train_loader)

Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 1] loss: 1.4179240465164185
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 2] loss: 1.4165242314338684
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 3] loss: 1.4110098282496135
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 4] loss: 1.3898365199565887
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 5] loss: 1.3925942659378052
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 6] loss: 1.3667033513387044
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 7] loss: 1.3668316773005895
Input shape: torch.Size([1, 128, 128]), Labels shape: t

Validating the model

In [143]:
def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on validation set: {100 * correct / total}%")

validate_model(model, val_loader)


Accuracy on validation set: 50.0%


Evaluating the model

In [144]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on test set: {100 * correct / total}%")

test_model(model, test_loader)


Accuracy on test set: 50.0%


Fine-tuning and hyperparameter optimization
Testing the model
Printing the guess (output)

In [146]:
# Convert the datasets into PyTorch datasets
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

# Define the data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Training the model
train_model(model, criterion, optimizer, train_loader, epochs=10)

# Validating the model
validate_model(model, val_loader)

# Testing the model
test_model(model, test_loader)

# Define a dictionary to map the numerical labels to their respective classes
LABEL_MAPPING = {
    0: 'Mild Demented',
    1: 'Moderate Demented',
    2: 'Non Demented',
    3: 'Very Mild Demented'
}

# Making predictions on a new image and printing the output class
def predict_image(image_path, model):
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted_class_idx = torch.max(output.data, 1)
        predicted_class_name = LABEL_MAPPING[predicted_class_idx.item()]

    return predicted_class_name

# Use the function like this:
predicted_class_name = predict_image('path_to_your_image.jpg', model)
print(f"The predicted class is: {predicted_class_name}")


C:\Users\Admin\AppData\Local\Temp\ipykernel_37556\318642730.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
C:\Users\Admin\AppData\Local\Temp\ipykernel_37556\318642730.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
C:\Users\Admin\AppData\Local\Temp\ipykernel_37556\318642730.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(X_test), torch.tenso

Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 1] loss: 1.0178512334823608
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 2] loss: 0.8771443367004395
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 3] loss: 1.9934523900349934
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 4] loss: 1.7495999932289124
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 5] loss: 1.5463229298591614
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 6] loss: 1.410652647415797
Input shape: torch.Size([1, 128, 128]), Labels shape: torch.Size([1])
Output shape: torch.Size([1, 4])
[1, 7] loss: 1.488125707421984
Input shape: torch.Size([1, 128, 128]), Labels shape: tor

KeyboardInterrupt: 